In [35]:
import torch
import torchvision
import torchvision.transforms as transforms

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import matplotlib.pyplot as plt
import numpy as np

from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter()

^C


AttributeError: type object 'h5py.h5.H5PYConfig' has no attribute '__reduce_cython__'

In [30]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 4

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [31]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.leaky_relu(self.conv1(x)))
        x = self.pool(F.leaky_relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.leaky_relu(self.fc1(x))
        x = F.leaky_relu(self.fc2(x))
        x = self.fc3(x)
        return x


model_leaky_relu = Net()

In [32]:
def train_model(optimizer, criterion, epochs, model):
  for epoch in range(epochs):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        writer.add_scalar("Loss/train", loss, epoch)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0

  print('Finished Training')
  writer.flush()
  writer.close()


In [33]:
def predict_test(model):
  correct = 0
  total = 0
  # since we're not training, we don't need to calculate the gradients for our outputs
  with torch.no_grad():
      for data in testloader:
          images, labels = data
          # calculate outputs by running images through the network
          outputs = model(images)
          # the class with the highest energy is what we choose as prediction
          _, predicted = torch.max(outputs.data, 1)
          total += labels.size(0)
          correct += (predicted == labels).sum().item()

  print(f'Accuracy of the network on the 10000 test images: {100 * correct // total} %')

### Use SGD as Optimizer



In [34]:
criterion = nn.CrossEntropyLoss()
optimizer_sgd = optim.SGD(model_leaky_relu.parameters(), lr=0.0001)
train_model(optimizer_sgd, criterion, 1, model_leaky_relu)
predict_test(model_leaky_relu)

NameError: name 'writer' is not defined

### Use ADAM as Optimizer


In [24]:
criterion = nn.CrossEntropyLoss()
optimizer_adam = optim.Adam(model_leaky_relu.parameters(), lr=0.0001)
train_model(optimizer_adam, criterion, 1, model_leaky_relu)
predict_test(model_leaky_relu)

[1,  2000] loss: 1.138
[1,  4000] loss: 1.136
[1,  6000] loss: 1.137
[1,  8000] loss: 1.121
[1, 10000] loss: 1.116
[1, 12000] loss: 1.115
Finished Training
Accuracy of the network on the 10000 test images: 58 %


## Use Tanh as Activation Function

In [25]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(torch.tanh(self.conv1(x)))
        x = self.pool(torch.tanh(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = torch.tanh(self.fc1(x))
        x = torch.tanh(self.fc2(x))
        x = self.fc3(x)
        return x


model_tanh = Net()

### Use SGD as Optimizer


In [26]:
criterion = nn.CrossEntropyLoss()
optimizer_sgd = optim.SGD(model_tanh.parameters(), lr=0.0001)
train_model(optimizer_sgd, criterion, 1, model_tanh)
predict_test(model_tanh)

[1,  2000] loss: 2.305
[1,  4000] loss: 2.302
[1,  6000] loss: 2.299
[1,  8000] loss: 2.297
[1, 10000] loss: 2.294
[1, 12000] loss: 2.288
Finished Training
Accuracy of the network on the 10000 test images: 17 %


### Use ADAM as Optimizer


In [27]:
criterion = nn.CrossEntropyLoss()
optimizer_adam = optim.Adam(model_tanh.parameters(), lr=0.0001)
train_model(optimizer_adam, criterion, 1, model_tanh)
predict_test(model_tanh)

[1,  2000] loss: 1.979
[1,  4000] loss: 1.764
[1,  6000] loss: 1.636
[1,  8000] loss: 1.598
[1, 10000] loss: 1.531
[1, 12000] loss: 1.515
Finished Training
Accuracy of the network on the 10000 test images: 46 %
